# Hosting on k8s

A containerized version of the documentation is hosted on the NCAR | CISL k8s cluster and can be viewed at [https://cloud-docs.k8s.ucar.edu/](https://cloud-docs.k8s.ucar.edu/). 

## Automatic Updates with [Argo CD](../K8s/argocd/argo-intro)

Whenever a change is made to the Helm chart for the documentation, [link to the Helm chart repo](https://github.com/NCAR/cisl-cloud/tree/main/configs/helm-charts/cloud-docs), Argo CD will automatically update the application to match the GitHub changes. If you have built a new container with an updated tag that you would like to deploy update the `webapp.container.image` value in the `values.yaml` file. You should also update the `Chart.yml` file with a new `version` number and `appVersion` value. After you have updated those 3 values push the changes to GitHub and Argo CD will update your application automatically when the next scheduled sync runs. Details on how to build a new container, either manually or with GitHub actions, can be found below. 

## Updating the image

By default any changes pushed to the `docs/_build/html/` directory will trigger a GitHub action that will build a new image, named `docs:latest` & `docs:<year-month-day>`, and push it to the cislcloudpilot Docker Hub repository. You should test any changes to this image locally before making changes to the hosted version with the following command: 

```
docker run --pull always -p 80:80 cislcloudpilot/docs:<year-month-day>
```

```{note}
The flag `--pull` has been set to always to ensure the latest version on Docker Hub is being run
```

If the changes look good it's time to push a new image tag to the container registry. This can be done 1 of 2 ways.

```{note}
Production versions of the image are tagged with the date it was released in the format year-month-day such as `2023-09-29`.  
```

### Locally tag and push

```{note}
In order to successfully push the image to Docker Hub you will need to login with the following command : `docker login --username=cislcloudpilot`. If you do not have the password contact Nick Cote to get access to the KeePass file that contains it.
```

Consider we just built a new image with the `:latest` tag to test our changes and we want to push the new image with an updated production tag back to the container registry. An example of how to do this is as follows:

```
docker image tag cislcloutpilot/docs:latest cislcloudpilot/docs:2023-09-29
docker image push cislcloudpilot/docs:2023-09-29
```

## Manually Upgrade Helm

```{note}
Any updates to the repository directory at this [link to docs Helm chart](https://github.com/NCAR/cisl-cloud/tree/main/configs/helm-charts/cloud-docs) will automatically trigger an application update via [Argo CD](../K8s/argocd/argo-intro). Manual upgrade via CLI is not required if you push changes directly to GitHub.
```

```{note}
This document assumes you have a kubeconfig file for the cluster and it is active. Documentation on how to accomplish this can be found [here](../K8s/Rancher/get-kubeconfig).
```

The cloud documentation was deployed on k8s using a helm chart. Instructions on how to accomplish this in more detail can be found [here](../K8s/Hosting/web-helm). Once you have a new docs image built, tagged, and pushed to Docker Hub it's time to update the helm chart hosted [here](https://github.com/NCAR/cisl-cloud/tree/main/configs/helm-charts/cloud-docs). More specifically we are going to change the `webapp.container.image` value that is set in the `values.yml` file and update the release information in `Chart.yaml`.

```{note}
The `appVersion` in `Chart.yaml` should directly match the image tag used in the container image. The proper `version` value should be incremented by 1 knowing that `version` is set to major.minor.patch. New content would be a change to the minor version number while any fixes that are implemented would be a change in the patch version number. The major version only changes when a very large change is made that ultimately affects how you interact with the image. 
```

With those changes in place to the helm chart files it's time to update the application on k8s. This can be accomplished by pushing the changes directly to GitHub and letting Argo CD update automatically, or manually by running the following command in the `helm-charts` directory :

```
helm upgrade -n ncote-test cloud-docs cloud-docs/
```

If everything runs successfully you should see output that looks like the following:
```
Release "cloud-docs" has been upgraded. Happy Helming!
NAME: cloud-docs
LAST DEPLOYED: Wed Aug  9 12:06:41 2023
NAMESPACE: ncote-test
STATUS: deployed
REVISION: 2
TEST SUITE: None
```

If there's an issue Helm should let you know what it is. Please address it and run the upgrade command again until it succeeds. If you have issues that you are unable to overcome please reach out to a team member for assistance. 